# Fine-tune a Mixtral-based LLM Prompt Generation model using `peft`, `transformers` and `bitsandbytes`

We can use the [Mosaic Instruct V3 Dataset](https://huggingface.co/datasets/mosaicml/instruct-v3) to fine-tune Mixtral to be able to generate LLM prompts based on LLM responses!

### Overview of PEFT and LoRA:

Based on some awesome new research [here](https://github.com/huggingface/peft), we can leverage techniques like PEFT and LoRA to train/fine-tune large models a lot more efficiently.

It can't be explained much better than the overview given in the above link:

```
Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of
pre-trained language models (PLMs) to various downstream applications without
fine-tuning all the model's parameters. Fine-tuning large-scale PLMs is often
prohibitively costly. In this regard, PEFT methods only fine-tune a small
number of (extra) model parameters, thereby greatly decreasing the
computational and storage costs. Recent State-of-the-Art PEFT techniques
achieve performance comparable to that of full fine-tuning.
```

### Install requirements

First, run the cells below to install the requirements:

In [ ]:
!pip install -qU flash-attn --no-build-isolation
!pip install transformers accelerate bitsandbytes peft -qU
!pip install -qU datasets
!pip install -qU trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 5.7 M

### Model loading

Here let's load the `Mixtral-8x7B` model!

In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bits_and_bytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_quant_type="nf4", # "NormalFloat 4-bit," - suited for normally distributed weights, such as those found in neural networks.
    bnb_4bit_use_double_quant=True
)

mixtral_7B = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bits_and_bytes_config,
    attn_implementation="flash_attention_2"
)

mixtral_tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00010-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00011-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub only has 149.67 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/blobs only has 149.67 MB free disk space.
  warnings.warn(


model-00012-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [ ]:
import torch.nn as nn
#not training the model, just training the adapter
for param in mixtral_7B.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability, these are more precision sensitive
    param.data = param.data.to(torch.float32)

mixtral_7B.gradient_checkpointing_enable()  # reduce number of stored activations
mixtral_7B.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
mixtral_7B.lm_head = CastOutputToFloat(mixtral_7B.lm_head)

In [ ]:
mixtral_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
import transformers
from torch.cuda.amp import autocast


text = "<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[\INST]"
inputs = mixtral_tokenizer(text, return_tensors="pt")

inputs = inputs.to('cuda')  # Assuming 'inputs' need to be on GPU
with autocast():
    outputs = mixtral_7B.generate(**inputs, max_new_tokens=150)

print(mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True))

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [ ]:
#next two cells from hugging face
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print(mixtral_7B)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, ou

In [ ]:
#New
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(mixtral_7B, peft_config)
print_trainable_parameters(mixtral_7B)

trainable params: 54525952 || all params: 23537127424 || trainable%: 0.23165933130991065


### Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the `load_dataset` method!

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mosaicml/instruct-v3")
dataset = dataset.filter(lambda x: x["source"] == "dolly_hhrlhf")
dataset

Generating train split:   0%|          | 0/56167 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6807 [00:00<?, ? examples/s]

Filter:   0%|          | 0/56167 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6807 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 34333
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 4771
    })
})

In [ ]:
dataset["train"] = dataset["train"].select(range(5_000))
dataset["test"] = dataset["test"].select(range(200))
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 200
    })
})

We want to put our data in the form [MIXTRAL expects](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1#instruction-format):

```
<s> [INST] Instruction [/INST] Model answer</s>
```

This way, we can prompt our model well and receive the responses we want!

This is what fine-tuning, and prompt-engineering, is really all about!

In [ ]:
dataset['train']

Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 5000
})

In [ ]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["response"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += system_message
  full_prompt += "\n" + input
  full_prompt += "[/INST]"
  full_prompt += response
  full_prompt += eos_token

  return full_prompt

In [ ]:
dataset["train"][0]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction\nWhat are different types of grass?\n\n### Response\n',
 'response': 'There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.',
 'source': 'dolly_hhrlhf'}

In [ ]:
create_prompt(dataset['train'][0])

'<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]What are different types of grass?</s>'

In [ ]:
model = prepare_model_for_kbit_training(mixtral_7B)
model = get_peft_model(mixtral_7B, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_ad_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 2,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=mixtral_tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss,Validation Loss
20,1.601100,1.542981
40,1.428900,1.422823
60,1.409700,1.389277
80,1.367600,1.381435
100,1.360800,1.378297


TrainOutput(global_step=100, training_loss=1.4680796909332274, metrics={'train_runtime': 511.2282, 'train_samples_per_second': 0.391, 'train_steps_per_second': 0.196, 'total_flos': 1.145886637817856e+17, 'train_loss': 1.4680796909332274, 'epoch': 0.43})

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
text = "<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]"
inputs = mixtral_tokenizer(text, return_tensors="pt")

outputs = merged_model.generate(
    **inputs,
    max_new_tokens=150,
    generation_kwargs={"repetition_penalty" : 1.7}
)
print(mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.
There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST] "List and describe the characteristics of some common species of grass, including Kentucky Bluegrass, Ryegrass, Fescues, and Bermuda grass."


## Share adapters on the 🤗 Hub

Make sure you have a Hugging Face account, and you have set up a read/write token!

More info here: https://huggingface.co/docs/hub/security-tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
mixtral_7B.push_to_hub("LLMPromptGen-AI", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/524M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bsbell21/LLMPromptGen-AI/commit/c98b5c7f2f6172e8375a8fb45931f181a3994d5e', commit_message='Upload MixtralForCausalLM', commit_description='', oid='c98b5c7f2f6172e8375a8fb45931f181a3994d5e', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below. However, given the size of Mixtral, two instances cannot exist simultaneously, so this code would only be run in another notebook if you wanted to pull the model:

In [ ]:
'''
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "{HUGGING_FACE_USER_NAME}/LLMPromptGen-AI"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
'''

'\nimport torch\nfrom peft import PeftModel, PeftConfig\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\n\npeft_model_id = f"{HUGGING_FACE_USER_NAME}/llm_instruction_generator"\nconfig = PeftConfig.from_pretrained(peft_model_id)\nmodel = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map=\'auto\')\ntokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)\n\n# Load the Lora model\nmodel = PeftModel.from_pretrained(model, peft_model_id)\n'

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference!

### Take it for a spin!

In [ ]:
def input_from_text(text):
  #formats LLM response text for Mixtral
  return "<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\n" + text + "[/INST]"

def get_instruction(text):
  #runs LLM response text through model and returns LLM Prompt
  inputs = mixtral_tokenizer(input_from_text(text), return_tensors="pt")

  outputs = merged_model.generate(
      **inputs,
      max_new_tokens=150,
      generation_kwargs={"repetition_penalty" : 1.7}
  )
  # print(mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True))
  result = mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True).split("[/INST]")[1]
  return result

### Example in Training Set

*Original LLM Response in Dataset:* "There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil."

*Original Predicted Prompt From Training Set*: "Generate a response describing the characteristics of four common types of grass: Kentucky Bluegrass, Ryegrass, Fescues, and Bermuda grass. Mention that Kentucky Bluegrass is the most common due to its quick and easy growth and soft texture. Describe Ryegrass as shiny and bright green. Characterize Fescues as dark green and shiny. And explain that Bermuda grass is harder and can grow in drier soil."

In [ ]:
text = "There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil."
get_instruction(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


' "List and describe the characteristics of some common species of grass, including Kentucky Bluegrass, Ryegrass, Fescues, and Bermuda grass."'


*New Response:* "Describe the characteristics of four common species of grass, including Kentucky Bluegrass, Ryegrass, Fescues, and Bermuda grass, in terms of their color, texture, and growth conditions."

### Example outside of Training Set

*LLM Response Example 1*: "American ex-pats are drawn to a variety of destinations around the world based on factors like climate, cost of living, culture, and ease of integration. Popular countries include Mexico, for its proximity to the United States and affordable cost of living; Spain, known for its rich culture and favorable climate; Portugal, with its beautiful landscapes and welcoming communities; Thailand, offering an affordable cost of living with vibrant culture; and Costa Rica, which is famed for its natural beauty and eco-friendly lifestyle. These countries not only provide a scenic change of pace but also boast strong ex-pat communities that help newcomers feel more at home."


In [ ]:
text = "American ex-pats are drawn to a variety of destinations around the world based on factors like climate, cost of living, culture, and ease of integration. Popular countries include Mexico, for its proximity to the United States and affordable cost of living; Spain, known for its rich culture and favorable climate; Portugal, with its beautiful landscapes and welcoming communities; Thailand, offering an affordable cost of living with vibrant culture; and Costa Rica, which is famed for its natural beauty and eco-friendly lifestyle. These countries not only provide a scenic change of pace but also boast strong ex-pat communities that help newcomers feel more at home."
get_instruction(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' "Generate a list of popular destinations for American expats, taking into account factors such as climate, cost of living, culture, and ease of integration."'

*LLM Response Example 2*: "The concept of cuteness in Pokémon is subjective, but three Pokémon frequently cited for their adorable qualities are Pikachu, Eevee, and Jigglypuff. "


In [ ]:
text = "The concept of cuteness in Pokémon is subjective, but three Pokémon frequently cited for their adorable qualities are Pikachu, Eevee, and Jigglypuff. "
get_instruction(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' "Generate an instruction that explains which three Pokémon are often considered cute by fans: Pikachu, Eevee, and Jigglypuff."'